# Using Numba for Dynamic Programming Problems
### by [Jason DeBacker](http://jasondebacker.com), June 2018
This Jupyter notebook was written using Python 3.6. 

[Numba](https://numba.pydata.org/) is a Python package useful for high performance computing.  It turns Python code (particularly Python code performing mathematical operations) into machine readable instructions using a just-in-time complier.

Numba has wide-ranging appilcations, but we'll focus here on an easy way to use Numba to speed up the solution to dynamic programming problems.  

Numba is sssawesome since it gives you the performance of C/Fortran while still using the simple syntax of Python.  In addition, it can be quite easy to implement.

## Our application

We solved our dynamic programming problem through value function iteration, a fixed point algorithm. At each iteration, we had to operate on the value function and find the optimal choice of control variable (e.g., $k'$) given our state vector (e.g., $k$).  We did this by applying the $max$ operator to an object we called `Vmat`.  `Vmat` was populated by looping over all combinations of $k$ and $k'$:

```
for i in range(sizek):  # loop over k
        for j in range(sizek):  # loop over k'
            Vmat[i, j] = e[i, j] + betafirm * V[j]
```

These loops are slow in Python. Let's see how slow...

In [72]:
# import packages
import numpy as np
import time

# define the size of the grid
sizek = 3000
betafirm = 0.96

# Initialize arrays
Vmat = np.zeros((sizek, sizek))
e = np.zeros((sizek, sizek))
V = np.zeros(sizek)

# Time our loop
start_time = time.clock()
for i in range(sizek):  # loop over k
        for j in range(sizek):  # loop over k'
            Vmat[i, j] = e[i, j] + betafirm * V[j]
print('Loop time (in seconds) = ', time.clock() - start_time)

Loop time (in seconds) =  5.918308000000003


Ok.  But how slow is this?  

Let's see by vectorizing our code (which should run faster in Python than running naive loops).

In [73]:
# Now we are timing the vectorized operation
start_time = time.clock()
VV = np.tile(np.reshape(V, (1, sizek)), (sizek, 1)) # here we've created a matrix for V(k') where each row is the same
Vmat = e + betafirm * VV
print('Loop time (in seconds) = ', time.clock() - start_time)

Loop time (in seconds) =  0.14862300000000062


That's a lot better!  So do we need Numba?  Good question.  It depends.  But vectorizing code can be difficult to read and write.  With large state spaces, it might even cause you memory issues as you end up carrying around objects that are larger than they otherwise need to be just so that dimensions line up for the array operations.

So how would Numba handle this?  First, we need to import Numba.  Second, we need to define a function that will use Numba.

In [74]:
# import packages
import numba

# define a function that will use Numba
@numba.jit
def VFI_loop(V, e, betafirm, sizek, Vmat):
    for i in range(sizek):  # loop over k
        for j in range(sizek):  # loop over k'
            Vmat[i, j] = e[i, j] + betafirm * V[j]

    return Vmat

The `@numba.jit` decorator before the function definition tells us that this function will call Numba to be compiles into machine readable code (though there are ways to use Numba for a function call.  `jit` stands for "just-in-time", as in the Numba just-in-time complier.

With this function defined, we can simply call it.  Let's see the speed here.

In [76]:
# Time our Numba function call
start_time = time.clock()
Vmat = VFI_loop(V, e, betafirm, sizek, Vmat)
print('Loop time (in seconds) = ', time.clock() - start_time)

Loop time (in seconds) =  0.017032999999997855


Depending on `sizek`, Numba does even better than our vectorized code (play around with this Notebook and find that "breakeven" point!).  As a plus, it was a lot easier to understand to read the code and see how the arrays are determined and where is element of the arrays is coming from.

When writing code, what matters it the total time it takes you to produce your desired result.  This includes both the time to write the code and the time it takes to execute the code.   Python usually dominates other languages on the former, but with Numba, it's often possible to write Python code and win on both counts.